In [1]:
#Imports
import torch, os
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
#LoadData
import yaml
import cv2
#Show dataset loading visualization
from tqdm import tqdm

#---------------------------------------------------------------------------------------------

#Serializer
def deserialize(file_name):
    with open(file_name, 'r') as f:
        obj =  yaml.load(f, Loader=yaml.FullLoader)
        return (obj['img_file_name'], obj['data'])
        
#Defining dataset class
class Faces_dataset(Dataset):
    def __init__(self,transform=None):
        self.data =[]#data of cv2 pics and labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img=self.data[idx][0]
        label=self.data[idx][1]
        if self.transform:
            img=self.transform(img)
        return (img, label)
    
    def add(self, item, label):
        self.data.append((item,label))

    #Create dataset function
def load_data(location,classid,dataset=Faces_dataset(ToTensor())):
    #dataset=dataset
    mainPic=None
    roiList=[]
    roi_size=(64,64)
    for yml in os.listdir(location):
        if yml.endswith(".yaml"):
            fname,roiList=deserialize(os.path.join(location, yml))
                #ROI
            if os.path.isfile(os.path.join(location, fname)):
                mainPic=cv2.imread(os.path.join(location, fname))
                for p1, p2 in roiList:
                    p1 = tuple([int(v) for v in p1])
                    p2 = tuple([int(v) for v in p2])
                    roi=mainPic[p1[1]:p2[1],p1[0]:p2[0]]
                    roi=cv2.resize(roi,roi_size)
                    dataset.add(roi,classid)
            mainPic=None   
    return dataset
    
#---------------------------------------------------------------------------------------------    

#Defining model class
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.fc1 = nn.Linear(16*14*14, 256)
        self.fc2 = nn.Linear(256, 25)
        self.fc3 = nn.Linear(25, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16*14*14)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

#Train function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

#Test function
def test(dataloader, model,loss_fn):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
#---------------------------------------------------------------------------------------------

#Defining device (cpu/gpu).
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

#Helpers
BASE_DIR=os.getcwd()
MODEL_DIR=os.path.join(BASE_DIR,'model')
#Model
model = NeuralNetwork().to(device)
USE_SAVED_MODEL=False
LOAD_PATH=os.path.join(MODEL_DIR,"model.pth")
SAVE_TRAINED_MODEL=True
SAVE_PATH=os.path.join(MODEL_DIR,"model.pth")
#Params
LOAD_DATA_TRAIN=True
LOCATION_DATA_TRAIN="data/train"
LOCATION_BAD_DATA_TRAIN="data/badtrain"
LOAD_DATA_TEST=True
LOCATION_DATA_TEST="data/test"
LOCATION_BAD_DATA_TEST="data/badtest"
#Hyperparams
BATCH_SIZE=32
EPOCHS=10
LOSS_FN = nn.CrossEntropyLoss() #Loss calculation function
LEARNING_RATE=1e-3
MOMENTUM=0.9
OPTIMIZER = torch.optim.SGD(model.parameters(), LEARNING_RATE, MOMENTUM) #Train function

#---------------------------------------------------------------------------------------------

    
#Load model to device and print it
if(USE_SAVED_MODEL):
    model.load_state_dict(torch.load(LOAD_PATH)) #load saved model
print(model)

#Training data
if(LOAD_DATA_TRAIN):
    training_data=load_data(LOCATION_DATA_TRAIN,1)
    load_data(LOCATION_BAD_DATA_TRAIN,0,training_data)
    
#Test data
if(LOAD_DATA_TEST):
    test_data=load_data(LOCATION_DATA_TEST,1)
    load_data(LOCATION_BAD_DATA_TEST,0,test_data)


#Data loaders
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

#Data loaders info (!)
'''
for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break
'''
    
    
#Training
for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, LOSS_FN, OPTIMIZER)
    test(test_dataloader, model,LOSS_FN)
print("Done!")
'''
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')
'''

#Save
if(SAVE_TRAINED_MODEL):
    torch.save(model.state_dict(), SAVE_PATH)
    print("Saved PyTorch Model State to \"{0}\"".format(SAVE_PATH))


#Defined classes of outputs
classes = [
    "Not face",
    "Face"
]

#Model in use (!)
'''
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')
'''
print("end")

Using cuda device
NeuralNetwork(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=3136, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=25, bias=True)
  (fc3): Linear(in_features=25, out_features=2, bias=True)
)
Epoch 1
-------------------------------
loss: 0.669755  [    0/   34]
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.039241 

Epoch 2
-------------------------------
loss: 0.667823  [    0/   34]
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.039001 

Epoch 3
-------------------------------
loss: 0.663749  [    0/   34]
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.038663 

Epoch 4
-------------------------------
loss: 0.657979  [    0/   34]
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.038246 

Epoch 5
-------------------------------
loss: 0.650885  [    0/   34]
Test Error: 
 Ac